In [1]:
from library import *

uobs = Observable('u', 1)
vobs = Observable('v', 1)
pobs = Observable('p', 0)
observables = [uobs, vobs, pobs]

### define all parameters and "make library"
terms = generate_terms_to(3, observables=observables, max_observables=3)

terms0 = []
terms1 = []
for term in terms:
    if term.rank==0:
        terms0.append(term)
    else:
        terms1.append(term)

[0, 0, 1, 0, 0] None 0
(0, 0, 0, 0, 0)
[0, 0, 0, 0, 0] CompList(in_list=[0, 0, 1, 0, 0]) 0
0 ([], (0,)) 0 []


TypeError: unhashable type: 'list'

In [ ]:
print(terms1)